# Задание 2

### Инициализация окружения
В качестве документации для задания 2 выступает данный документ Jupyter Notebook. В нём записаны все скрипты и SQL-запросы, касающиеся пунктов 2.1 и 2.2.

venv для данного задания и задания 3 одинаковый, поэтому на данном этапе можно задействовать некоторые из необходимых далее библиотек.

In [ ]:
%pip install sqlalchemy psycopg2-binary pandas

Далее следует создать соединение к базе данных, через которое будут протестированы все последующие запросы.
Необходимые данные для соединения с БД должны браться хотя бы из .env-файла, но в рамках Jupyter Notebook все параметры внесены в отдельный словарь.

In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Параметры подключения к БД
DB_CONFIG = {
    'drivername': 'postgresql',
    'username': 'postgres',
    'password': 'postgres',
    'host': 'localhost',
    'port': '5432',
    'database': 'main_db'
}

# Создаём движок SQLAlchemy
engine = create_engine(
            f"{DB_CONFIG['drivername']}://{DB_CONFIG['username']}:{DB_CONFIG['password']}@"
            f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
        )
# Создаём соединение с БД на основе движка
db_connection = engine.connect()
db_connection.closed # Если выведен False, то соединение настроено

False

### Задание 2.1.
Для данного задания необходимо получить информацию о сумме товаров, заказанных под каждого клиента.

Получение суммы реализовано через последовательность запросов JOIN вместе с функцией COALESCE, позволяющей вывести сумму 0 в случае отсутствия заказов у данного клиента.
Вместо обычного JOIN используется LEFT JOIN для включения в вывод клиентов, которые ещё не сделали ни одного заказа.

In [2]:
task_2_1_query = """
SELECT 
    c.name AS client_name,
    COALESCE(SUM(og.amount), 0) AS total_amount
FROM Clients c
LEFT JOIN Orders o ON o.client_id = c.id
LEFT JOIN Ordered_goods og ON og.order_id = o.id
LEFT JOIN Goods g ON g.id = og.good_id
GROUP BY c.id, c.name
ORDER BY total_amount DESC;
"""

# Выполняем запрос на получения данных из БД
df = pd.read_sql(task_2_1_query, engine)
df

,client_name,total_amount
0,Козлова Ирина Петровна,29
1,Кузнецов Алексей Викторович,29
2,Захаров Сергей Викторович,29
3,Петров Петр Петрович,23
4,Смирнова Ольга Владимировна,20
5,Иванов Иван Иванович,17
6,Волков Андрей Сергеевич,13
7,Сидорова Мария Сергеевна,8
8,Николаева Екатерина Александровна,2
9,Федоров Дмитрий Николаевич,0


При этом может потребоваться информация о суммарной стоимости всех заказанных товарах, которое получается простой заменой выражения внутри функции SUM.

In [15]:
# Заменяем выражение внутри функции SUM
sum_price_query = task_2_1_query.replace("SUM(og.amount)", "SUM(g.price * og.amount)")

# Заменяем все упоминания total_amount на total_cost
sum_price_query = sum_price_query.replace("total_amount", "total_cost")

df = pd.read_sql(sum_price_query, engine)
df

,client_name,total_cost
0,Козлова Ирина Петровна,1817971.9
1,Кузнецов Алексей Викторович,1794971.9
2,Захаров Сергей Викторович,1731471.9
3,Петров Петр Петрович,1335977.9
4,Смирнова Ольга Владимировна,1187480.9
5,Иванов Иван Иванович,830484.8
6,Волков Андрей Сергеевич,804987.9
7,Сидорова Мария Сергеевна,449992.0
8,Николаева Екатерина Александровна,128998.0
9,Федоров Дмитрий Николаевич,0.0


### Задание 2.2.
Необходимо написать запрос на поиск количества дочерних элементов первого уровня вложенности для категорий номенклатуры.

Поскольку при выполнении данного задания используется PostgreSQL с расширением ltree, в запросах применяются соответствующие инструменты поиска элементов по дереву. Хотя это решение не универсально среди всех реляционных СУБД, в данных условиях оно наиболее оптимальное.

***Комментарии к запросу***
1. *LEFT JOIN* используется, опять же, для вывода элементов первого уровня без дочерних подэлементов.
2. Условие "*ON c2.path <@ c1.path*" возвращает True в случае иерархической принадлежности текущего элемента из таблицы *Catalogue c2* данному корневому элементу из *c1*.
3. Условие "*AND nlevel(c2.path) = nlevel(c1.path) + 1*" возвращает True, если текущий элемент из *c2* является подэлементом первого уровня вложенности для данной категории из *c1*.

In [23]:
task_2_2_query = """
SELECT 
    c1.id,
    c1.name,
    COUNT(c2.id) AS num_of_children
FROM Catalogue c1
LEFT JOIN Catalogue c2
          ON c2.path <@ c1.path
          AND nlevel(c2.path) = nlevel(c1.path) + 1
GROUP BY c1.id, c1.name
ORDER BY num_of_children DESC;
"""

df = pd.read_sql(task_2_2_query, engine)
df

,id,name,num_of_children
0,1,Бытовая техника,3
1,8,Ноутбуки,2
2,2,Компьютеры,2
3,4,Холодильники,2
4,11,Моноблоки,0
5,9,17',0
6,5,однокамерные,0
7,3,Стиральные машины,0
8,10,19',0
9,6,двухкамерные,0


### Задание 2.3.1.
Требуется написать VIEW для вывода топ-5 самых покупаемых товаров по количеству штук в заказах.

Для отчёта названия атрибутов итоговой таблицы приведены на русском.
Категорию 1-го уровня для данного товара с помощью ltree получить достаточно просто: необходимо разделить всю строку по знаку '.' и взять первый элемент массива как наименование корневой категории.
Однако, поскольку изначально в path для краткости хранится последовательность ID, надо предварительно выделить из неё конкретное наименование категории.

Поскольку требуется выделить топ-5 элементов, в данном запросе для оптимизации используется ограничение *LIMIT 5*. Однако если на одном из мест будет несколько претендентов, всё равно выводятся только 5 позиций, что может быть нежелательно в некоторых случаях.

In [5]:
# Для выделения названия категории пришлось добавить новый JOIN
# Так как на уровне проектирования индексы добавлены не были,
# выполнение данного VIEW на высоконагруженной системе может сильно замедлить работу БД

view_task_query = """
CREATE OR REPLACE VIEW top5_purchased_goods AS
SELECT 
    g.name AS Наименование_товара,
    root_catalogue.name AS Категория_1_го_уровня,
    SUM(og.amount) AS Общее_количество_проданных_штук
FROM Goods g
JOIN Ordered_goods og ON g.id = og.good_id
JOIN Catalogue c ON g.catalogue_id = c.id
JOIN Catalogue root_catalogue ON root_catalogue.id = (split_part(c.path::text, '.', 1))::integer
GROUP BY g.id, g.name, root_catalogue.name
ORDER BY Общее_количество_проданных_штук DESC
LIMIT 5;
"""

Устранить данный недостаток можно, используя оконную функцию *DENSE RANK()* (функция *RANK* может быть нежелательна, так как разрывает порядок ранжирования), однако это несколько усложнит и затормозит VIEW-запрос, в связи с чем его следует применять только при явном объявлении об этом в бизнес-требованиях или ТЗ.

In [ ]:
extended_view_task_query = """
CREATE OR REPLACE VIEW top5_purchased_goods AS
WITH goods_sales AS (
    SELECT 
        g.id AS goods_id,
        g.name AS goods_name,
        (split_part(c.path::text, '.', 1))::integer AS root_category_id,
        SUM(og.amount) AS total_sold
    FROM Goods g
    LEFT JOIN Ordered_goods og ON g.id = og.good_id
    LEFT JOIN Catalogue c ON g.catalogue_id = c.id
    GROUP BY g.id, g.name, c.path
),
top5 AS (
    SELECT 
        goods_id,
        goods_name,
        root_category_id,
        total_sold,
        DENSE_RANK() OVER (ORDER BY total_sold DESC) AS position
    FROM goods_sales
)
SELECT 
    t5.goods_name AS Наименование_товара,
    cat.name AS Категория_1_го_уровня,
    t5.total_sold AS Общее_количество_проданных_штук
FROM top5 t5
LEFT JOIN Catalogue cat ON t5.root_category_id = cat.id
WHERE t5.position <= 5
ORDER BY t5.total_sold DESC, t5.goods_name;
"""

Соответственно, поскольку в ТЗ явно не прописано использование более сложной версии, в дальнейшем предполаегется использование варианта с применением *LIMIT 5*.

In [16]:
from sqlalchemy import text

db_connection.execute(text(view_task_query))
db_connection.commit()

df = pd.read_sql("SELECT * FROM top5_purchased_goods;", engine)
df

,Наименование_товара,Категория_1_го_уровня,Общее_количество_проданных_штук
0,"Ноутбук Lenovo 17""",Компьютеры,16
1,Телевизор LG,Бытовая техника,15
2,Телевизор Sony,Бытовая техника,14
3,"Ноутбук Dell 17""",Компьютеры,14
4,Холодильник Samsung,Бытовая техника,13


### Задание 2.3.2.
Необходимо проанализировать созданный запрос и саму структуру БД, а также оптимизировать её под увеличивающуюся нагрузку.

Для начала следует определить оптимальность выполнения запроса из задания 2.3.1.

*Примечание.* Вне Jupyter Notebook для *EXPLAIN ANALYZE* необходимо использовать транзакцию с командой *ROLLBACK*, однако здесь при запросе на получение данных можно обойтись простым исполнением команды. В случае возникновения ошибки запрос *ROLLBACK;* возможно запустить в любом окне с кодом.

In [81]:
analyze_query = """
EXPLAIN ANALYZE SELECT * FROM top5_purchased_goods;
"""

report = db_connection.execute(text(analyze_query)).fetchall()
report

[('Subquery Scan on top5_purchased_goods  (cost=52.94..53.01 rows=5 width=644) (actual time=0.231..0.234 rows=5 loops=1)',),
 ('  ->  Limit  (cost=52.94..52.96 rows=5 width=648) (actual time=0.231..0.233 rows=5 loops=1)',),
 ('        ->  Sort  (cost=52.94..53.32 rows=150 width=648) (actual time=0.230..0.231 rows=5 loops=1)',),
 ('              Sort Key: (sum(og.amount)) DESC',),
 ('              Sort Method: top-N heapsort  Memory: 26kB',),
 ('              ->  HashAggregate  (cost=48.95..50.45 rows=150 width=648) (actual time=0.220..0.223 rows=15 loops=1)',),
 ('                    Group Key: g.id, root_catalogue.name',),
 ('                    Batches: 1  Memory Usage: 40kB',),
 ('                    ->  Hash Join  (cost=33.20..47.83 rows=150 width=644) (actual time=0.042..0.188 rows=150 loops=1)',),
 ("                          Hash Cond: ((split_part((c.path)::text, '.'::text, 1))::integer = root_catalogue.id)",),
 ('                          ->  Hash Join  (cost=16.68..30.80 rows

Также следует выполнить запрос *EXPLAIN BUFFERS* для проверки использования ОЗУ.

In [103]:
analyze_query = """
EXPLAIN (ANALYZE,BUFFERS) SELECT * FROM top5_purchased_goods;
"""

report = db_connection.execute(text(analyze_query)).fetchall()
report

[('Subquery Scan on top5_purchased_goods  (cost=52.94..53.01 rows=5 width=644) (actual time=0.239..0.243 rows=5 loops=1)',),
 ('  Buffers: shared hit=33',),
 ('  ->  Limit  (cost=52.94..52.96 rows=5 width=648) (actual time=0.239..0.241 rows=5 loops=1)',),
 ('        Buffers: shared hit=33',),
 ('        ->  Sort  (cost=52.94..53.32 rows=150 width=648) (actual time=0.238..0.240 rows=5 loops=1)',),
 ('              Sort Key: (sum(og.amount)) DESC',),
 ('              Sort Method: top-N heapsort  Memory: 26kB',),
 ('              Buffers: shared hit=33',),
 ('              ->  HashAggregate  (cost=48.95..50.45 rows=150 width=648) (actual time=0.228..0.232 rows=15 loops=1)',),
 ('                    Group Key: g.id, root_catalogue.name',),
 ('                    Batches: 1  Memory Usage: 40kB',),
 ('                    Buffers: shared hit=33',),
 ('                    ->  Hash Join  (cost=33.20..47.83 rows=150 width=644) (actual time=0.044..0.196 rows=150 loops=1)',),
 ("                  

В результате выполнения двух указанных запросов можно сделать следующие выводы:
1. Во время выполнения VIEW-запроса приходится три раза полностью сканировать таблицы: *Seq scan* дважды используется для таблицы *Catalogue* и один раз - для таблицы *Ordered_goods*.
2. При этом поиск по таблице *Goods* уже производится по индексу на Primary Key *goods_pkey* через псевдоним g.
3. В ходе сравнения первого элемента атрибута path с root_catalogue.id происходит множество операций преобразования. При большом количестве товаров это может привести к лишним вычислениям, хотя при этом они не зависят напрямую от количества заказов.
4. Повторное исполнение запроса с BUFFERS не приводит к увеличению числа совпадений, а количество Batches везде равно единице. Следовательно, текущий объём ОЗУ полностью покрывает потребности кэша СУБД.
5. Хотя данный отчёт формируется только для учёта статистики за последний месяц, VIEW будет пересчитываться при каждом обращении к нему, из-за чего происходит лишнее выполнение запроса при обращении к нему несколько раз за месяц.

Соответственно, для оптимизации хотя бы данного запроса следует выполнить следующие шаги:
- Добавить индексы в таблицы *Catalogue* и *Ordered_goods* по соответствующим атрибутам: для *Catalogue* необходимо убедиться в создании индекса для *path*, а в *Ordered_goods* - для всех её FK. Но в целом желательно добавить индексы на всех FK, где они отсутствуют.
- По возможности оптимизировать процесс преобразования типов *path* в *id*.
- Добавить к данному VIEW идентичный MATERIALIZED VIEW, чтобы запрос не перевычислялся при каждом обращении к нему. При этом изначальный VIEW имеет смысл оставить как есть на случай, если понадобится единоразовая актуальная сводка.

In [40]:
# Добавляем запросы на создание индексов
index_queries = [
    # Индекс на FK Ordered_goods -> Goods
    "CREATE INDEX IF NOT EXISTS idx_ordered_goods_good_id ON Ordered_goods(good_id);",
    # Индекс на FK Ordered_goods -> Orders
    "CREATE INDEX IF NOT EXISTS idx_ordered_goods_order_id ON Ordered_goods(order_id);",
    # Индекс на FK Goods -> Catalogue
    "CREATE INDEX IF NOT EXISTS idx_goods_catalogue_id ON Goods(catalogue_id);",
    # Индекс по nlevel(path) для проверки усорвия корневого элемента nlevel(path) = 1
    "CREATE INDEX idx_catalogue_nlevel_path ON Catalogue (nlevel(path));",
    # Два различных индекса на path; GIST ускоряет операции со строками, а BTREE - с проверкой равенства
    "CREATE INDEX IF NOT EXISTS idx_catalogue_path_gist ON Catalogue USING GIST (path);",
    "CREATE INDEX IF NOT EXISTS idx_catalogue_path_btree ON Catalogue USING BTREE (path);",
    # Индекс на FK Orders -> Clients
    "CREATE INDEX IF NOT EXISTS idx_orders_client_id ON Orders(client_id);"
]

for index in index_queries:
    db_connection.execute(text(index))
db_connection.commit()

In [ ]:
# Создаём улучшенный MATERIALIZED VIEW
mat_view_query = """
CREATE MATERIALIZED VIEW top5_monthly_purchased_goods AS
SELECT 
    g.name AS Наименование_товара,
    root_cat.name AS Категория_1_го_уровня,
    SUM(og.amount) AS Общее_количество_проданных_штук
FROM Goods g
JOIN Ordered_goods og ON g.id = og.good_id
JOIN Catalogue c ON g.catalogue_id = c.id
JOIN Catalogue root_cat ON c.path <@ root_cat.path
WHERE nlevel(root_cat.path) = 1
GROUP BY g.id, g.name, root_cat.name
ORDER BY Общее_количество_проданных_штук DESC
LIMIT 5;
"""

# Инициализируем MAT. VIEW для дальнейшего использования
# db_connection.execute(text("DROP MATERIALIZED VIEW top5_monthly_purchased_goods;")) # На случай пересоздания MV
db_connection.execute(text(mat_view_query))
db_connection.commit()

In [ ]:
# В рамках тестирования можно обновлять MAT. VIEW перед использованием
db_connection.execute(text("REFRESH MATERIALIZED VIEW top5_monthly_purchased_goods;"))
db_connection.commit()

df = pd.read_sql("SELECT * FROM top5_monthly_purchased_goods;", engine)
df

,Наименование_товара,Категория_1_го_уровня,Общее_количество_проданных_штук
0,"Ноутбук Lenovo 17""",Компьютеры,16
1,Телевизор LG,Бытовая техника,15
2,Телевизор Sony,Бытовая техника,14
3,"Ноутбук Dell 17""",Компьютеры,14
4,Холодильник Samsung,Бытовая техника,13


In [ ]:
# Анализ выполнения MATERIALIZED VIEW
analyze_query = """
EXPLAIN (ANALYZE,BUFFERS) SELECT * FROM top5_monthly_purchased_goods;
"""

report = db_connection.execute(text(analyze_query)).fetchall()
report

[('Seq Scan on top5_monthly_purchased_goods  (cost=0.00..11.20 rows=120 width=644) (actual time=0.006..0.007 rows=5 loops=1)',),
 ('  Buffers: shared hit=1',),
 ('Planning Time: 0.027 ms',),
 ('Execution Time: 0.015 ms',)]

In [46]:
# Анализ выполнения самого запроса
analyze_query = """
EXPLAIN (ANALYZE,BUFFERS) SELECT 
    g.name AS Наименование_товара,
    root_cat.name AS Категория_1_го_уровня,
    SUM(og.amount) AS Общее_количество_проданных_штук
FROM Goods g
JOIN Ordered_goods og ON g.id = og.good_id
JOIN Catalogue c ON g.catalogue_id = c.id
JOIN Catalogue root_cat ON c.path <@ root_cat.path
WHERE nlevel(root_cat.path) = 1
GROUP BY g.id, g.name, root_cat.name
ORDER BY Общее_количество_проданных_штук DESC
LIMIT 5;
"""

report = db_connection.execute(text(analyze_query)).fetchall()
report

[('Limit  (cost=4.95..4.95 rows=2 width=648) (actual time=0.143..0.144 rows=5 loops=1)',),
 ('  Buffers: shared hit=52',),
 ('  ->  Sort  (cost=4.95..4.95 rows=2 width=648) (actual time=0.142..0.143 rows=5 loops=1)',),
 ('        Sort Key: (sum(og.amount)) DESC',),
 ('        Sort Method: top-N heapsort  Memory: 26kB',),
 ('        Buffers: shared hit=52',),
 ('        ->  GroupAggregate  (cost=4.90..4.94 rows=2 width=648) (actual time=0.112..0.136 rows=15 loops=1)',),
 ('              Group Key: g.id, root_cat.name',),
 ('              Buffers: shared hit=52',),
 ('              ->  Sort  (cost=4.90..4.90 rows=2 width=644) (actual time=0.108..0.114 rows=150 loops=1)',),
 ('                    Sort Key: g.id, root_cat.name',),
 ('                    Sort Method: quicksort  Memory: 37kB',),
 ('                    Buffers: shared hit=52',),
 ('                    ->  Nested Loop  (cost=0.28..4.89 rows=2 width=644) (actual time=0.022..0.077 rows=150 loops=1)',),
 ('                       

In [45]:
# Проверка выполнения запроса без Seq Scan
db_connection.execute(text("SET enable_seqscan TO off;"))
db_connection.commit()

report = db_connection.execute(text(analyze_query)).fetchall()
db_connection.execute(text("SET enable_seqscan TO on;"))
db_connection.commit()

report

[('Limit  (cost=18.85..18.85 rows=2 width=648) (actual time=0.144..0.146 rows=5 loops=1)',),
 ('  Buffers: shared hit=55',),
 ('  ->  Sort  (cost=18.85..18.85 rows=2 width=648) (actual time=0.143..0.145 rows=5 loops=1)',),
 ('        Sort Key: (sum(og.amount)) DESC',),
 ('        Sort Method: top-N heapsort  Memory: 26kB',),
 ('        Buffers: shared hit=55',),
 ('        ->  GroupAggregate  (cost=18.80..18.84 rows=2 width=648) (actual time=0.114..0.138 rows=15 loops=1)',),
 ('              Group Key: g.id, root_cat.name',),
 ('              Buffers: shared hit=55',),
 ('              ->  Sort  (cost=18.80..18.80 rows=2 width=644) (actual time=0.109..0.116 rows=150 loops=1)',),
 ('                    Sort Key: g.id, root_cat.name',),
 ('                    Sort Method: quicksort  Memory: 37kB',),
 ('                    Buffers: shared hit=55',),
 ('                    ->  Nested Loop  (cost=0.55..18.79 rows=2 width=644) (actual time=0.024..0.078 rows=150 loops=1)',),
 ('              

Из анализа запросов видно, что индексы настроены так, как надо и могут быть использованы планировщиком. Однако, в связи с малым количеством тестовых данных, PostgreSQL предпочитает Seq Scan вместо Index Scan, так как он при такой выборке работает слегка быстрее.

Анализ MATERIALIZE VIEW показывает, что при обращении к заранее сформированному отчёту скорость выполнения запроса может быть в 10 раз больше, чем при постоянном использовани одного лишь VIEW-запроса. Однако его требуется ежемесячно обновлять, для чего понадобится установить отдельную службу в cron, Apache Airflow или любой другой платформе. Запрос *REFRESH* приведён выше, рядом со скриптом запуска *SELECT * FROM top5_monthly_...*

### Оптимизация БД под высокие нагрузки
Хотя часть запросов стала выполняться лучше, данной архитектуры недостаточно для обеспечения выполнения запросов с потоком в 1000 заказов в день. Поэтому необходимо ввести некоторые изменения в таблицы *Orders* и *Ordered_Goods*:

В других СУБД имело бы смысл создание staging-таблиц и замена партиций на заполненную к концу дня staging-таблицу.
Однако в PostgreSQL более целесообразно применять партиционирование по хэш-значению. Если ввести партиционирование по дате заказов (условно, добавить новый атрибут), то тогда пришлось бы гарантировать уникальность именно атрибута date, а не составного ключа (id, date), что не имеет смысла в случае 1000 заказов в день.

Таблица *Ordered_goods* будет слишком перегружена, чтобы её разделять по дате, в связи с чем самым эффективным способом партиционирования двух данных таблиц является хэш-партиционирование. Для генерации партиций на основе новых таблиц можно создать собственные функции, которые будут автоматически распределять партиции по остаткам деления, либо использовать pg_partman.

*Примечание.* Поскольку в ТЗ это явно не описывалось, на этапе проектирования в таблицу *Ordered_goods* не была внесена информация о стоимости товара на момент покупки и дате заказа. В идеальном случае эту возможность следовало бы уточнить у заказчика или аналитика, чтобы при изменении стоимости товара не приходилось менять уже рабочую БД.

In [ ]:
# Далее приведены SQL-запросы, определяющие новые таблицы
"""
CREATE TABLE Ordered_goods (
    order_id BIGINT  NOT NULL REFERENCES Orders(id) ON DELETE CASCADE,
    good_id  INTEGER NOT NULL REFERENCES Goods(id) ON DELETE CASCADE,
    amount   INTEGER CHECK (amount > 0),
    PRIMARY KEY (order_id, good_id)
) PARTITION BY HASH (order_id, good_id);


CREATE OR REPLACE FUNCTION create_hash_partitions()
RETURNS void AS $$
DECLARE
    i INT;
    partition_name TEXT;
    create_partition_query TEXT;
BEGIN
    FOR i IN 0..999 LOOP
        partition_name := 'ordered_goods_part_' || lpad(i::text, 4, '0');
        
        create_partition_query := 
            'CREATE TABLE ' || partition_name || 
            ' PARTITION OF Ordered_goods FOR VALUES WITH (MODULUS 1000, REMAINDER ' || i || ')';
        
        EXECUTE create_partition_query;
    END LOOP;
END;
$$ LANGUAGE plpgsql;


CREATE TABLE Orders (
    id        BIGSERIAL PRIMARY KEY,
    client_id INTEGER NOT NULL REFERENCES Clients(id) ON DELETE CASCADE
) PARTITION BY HASH (id);


CREATE OR REPLACE FUNCTION create_orders_partitions()
RETURNS void AS $$
DECLARE
    i INTEGER;
    partition_name TEXT;
    sql TEXT;
BEGIN
    -- Создаем 100 партиций с использованием хэш-партиционирования
    FOR i IN 0..99 LOOP
        partition_name := 'orders_part_' || lpad(i::text, 3, '0');
        
        -- Формируем SQL запрос для создания партиции
        sql := format(
            'CREATE TABLE IF NOT EXISTS %I PARTITION OF Orders FOR VALUES WITH (MODULUS 100, REMAINDER %s)',
            partition_name, i
        );
        
        -- Выполняем создание партиции
        EXECUTE sql;
        
        -- Добавляем комментарий к партиции для удобства
        EXECUTE format('COMMENT ON TABLE %I IS %L', 
                      partition_name, 
                      'Partition of Orders table for hash remainder ' || i);
    END LOOP;
    
    RAISE NOTICE '100 partitions for Orders table created successfully';
END;
$$ LANGUAGE plpgsql;
"""

Dumpfile изменённой БД приведён в той же директории, что и данная документация. Поскольку схема координально не изменялась, её ER-диаграмма осталась такой же.

In [100]:
# Завершаем сеанс
db_connection.close()
engine.dispose()